In [ ]:
# binder https://hub.gke.mybinder.org/user/blychs-climatologia_fisica-dckeuhha/notebooks/practica2.ipynb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
import pandas as pd
import xarray as xr
import copy
from metpy.units import units
import metpy.calc as mpcalc
import metpy.constants as mpconsts
import datetime as dt
import calendar
import seaborn as sns
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib as mpl
#from funciones_meteo import load_file, ploteo_general, calculate_h

In [ ]:
class Celda():
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

In [ ]:
def area_banda(lat1, lat2):
    """Calcula el area de una banda de latitudes"""
    r = 6371000
    cap = lambda x: 2 * np.pi * (6371000**2) * np.abs(np.sin(np.deg2rad(x)) - 1)
    a1 = cap(lat1)
    a2 = cap(lat2)
    return np.abs(cap(lat1) - cap(lat2))


def area_celda(lat, delta_lat=2.5, delta_lon=2.5):
    """Calcula el area de una celda de lat
    en base a una celda central"""
    return (np.abs(delta_lon) / 360 * area_banda(lat-(delta_lat/2),lat+(delta_lat/2)))


def regrid(xarray):
    """Lleva a una grilla con los puntos entre 2 puntos
    adyacentes de la original que tenemos"""
    new_grid = np.linspace(-88.75, 88.75, len(xarray.lat) - 1)
    return xarray.interp(lat=new_grid)


todas_celdas = np.arange(-90, 92.5, 2.5)
area_tierra = area_banda(-90, 90)
print(np.sum(area_celda(todas_celdas) * 144/ area_tierra))
print(str((area_banda(-90, 90))/10**6))
print(np.sum(area_celda(todas_celdas) * 144/ 10**6))
for i in range(90, -90, -10):
    print('Area banda ' + str(i) + '-' + str(i+10) + ' = '+ str(area_banda(i, i -10) / 10**12))


In [ ]:
# Load files CMIP5

pr_c5_hist = xr.open_mfdataset('NorESM1-M/pr_Amon_NorESM1-M_historical_r*i1p1_197601-200512_2.5_anu.nc',
                              combine='nested',
                              concat_dim='r').metpy.parse_cf('pr')
pr_c5_ens = pr_c5_hist.mean(dim='r', keep_attrs=True)
pr_c5_26_close = xr.open_dataset('NorESM1-M/pr_Amon_NorESM1-M_rcp26_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('pr')
pr_c5_26_far = xr.open_dataset('NorESM1-M/pr_Amon_NorESM1-M_rcp26_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('pr')
pr_c5_85_close = xr.open_dataset('NorESM1-M/pr_Amon_NorESM1-M_rcp85_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('pr')
pr_c5_85_far = xr.open_dataset('NorESM1-M/pr_Amon_NorESM1-M_rcp85_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('pr')
# """"""""""""""""""""""""""""""""""""""""""
ev_c5_hist = xr.open_mfdataset('NorESM1-M/evspsbl_Amon_NorESM1-M_historical_r1i1p1_197601-200512_2.5_anu.nc',
                               combine='nested',
                               concat_dim='r').metpy.parse_cf('evspsbl')
ev_c5_ens = ev_c5_hist.mean(dim='r', keep_attrs=True)
ev_c5_26_close = xr.open_dataset('NorESM1-M/evspsbl_Amon_NorESM1-M_rcp26_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('evspsbl')
ev_c5_26_far   = xr.open_dataset('NorESM1-M/evspsbl_Amon_NorESM1-M_rcp26_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('evspsbl')
ev_c5_85_close = xr.open_dataset('NorESM1-M/evspsbl_Amon_NorESM1-M_rcp85_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('evspsbl')
ev_c5_85_far   = xr.open_dataset('NorESM1-M/evspsbl_Amon_NorESM1-M_rcp85_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('evspsbl')
# """"""""""""""""""""""""""""""""""""""""""
ep_c5_hist = xr.open_mfdataset('NorESM1-M/etp_Amon_NorESM1-M_historical_r*i1p1_197601-200512_2.5_anu.nc',
                               combine='nested',
                               concat_dim='r').metpy.parse_cf('etp')
ep_c5_ens = ep_c5_hist.mean(dim='r', keep_attrs=True)
ep_c5_26_close = xr.open_dataset('NorESM1-M/etp_Amon_NorESM1-M_rcp26_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('etp')
ep_c5_26_far   = xr.open_dataset('NorESM1-M/etp_Amon_NorESM1-M_rcp26_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('etp')
ep_c5_85_close = xr.open_dataset('NorESM1-M/etp_Amon_NorESM1-M_rcp85_r1i1p1_202001-204912_2.5_anu.nc'
                                ).metpy.parse_cf('etp')
ep_c5_85_far   = xr.open_dataset('NorESM1-M/etp_Amon_NorESM1-M_rcp85_r1i1p1_207001-209912_2.5_anu.nc'
                                ).metpy.parse_cf('etp')
# """"""""""""""""""""""""""""""""""""""""""""

t_c5_hist = xr.open_mfdataset('NorESM1-M/tas_Amon_NorESM1-M_historical_r*i1p1_197601-200512_2.5_anu.nc',
                              combine='nested',
                              concat_dim='r').metpy.parse_cf('tas')
t_c5_ens = t_c5_hist.mean(dim='r', keep_attrs=True)
t_c5_26_close = xr.open_dataset('NorESM1-M/tas_Amon_NorESM1-M_rcp26_r1i1p1_202001-204912_2.5_anu.nc'
                               ).metpy.parse_cf('tas')
t_c5_26_far   = xr.open_dataset('NorESM1-M/tas_Amon_NorESM1-M_rcp26_r1i1p1_207001-209912_2.5_anu.nc'
                               ).metpy.parse_cf('tas')
t_c5_85_close = xr.open_dataset('NorESM1-M/tas_Amon_NorESM1-M_rcp85_r1i1p1_202001-204912_2.5_anu.nc'
                               ).metpy.parse_cf('tas')
t_c5_85_far   = xr.open_dataset('NorESM1-M/tas_Amon_NorESM1-M_rcp85_r1i1p1_207001-209912_2.5_anu.nc'
                               ).metpy.parse_cf('tas')
# """"""""""""""""""""""""""""""""""""""
q_c5_hist = xr.open_mfdataset('NorESM1-M/huss_historical_r*_anu.nc',
                            combine='nested',
                            concat_dim='r').metpy.parse_cf('huss')
q_c5_ens = q_c5_hist.mean(dim='r', keep_attrs=True)
q_c5_26_close = xr.open_dataset('NorESM1-M/huss_Amon_NorESM1-M_rcp26_r1_202001-204912_2.5_anu.nc'
                               ).metpy.parse_cf('huss')
q_c5_26_far   = xr.open_dataset('NorESM1-M/huss_Amon_NorESM1-M_rcp26_r1_207001-209912_2.5_anu.nc'
                               ).metpy.parse_cf('huss')
q_c5_85_close = xr.open_dataset('NorESM1-M/huss_Amon_NorESM1-M_rcp85_r1_202001-204912_2.5_anu.nc'
                               ).metpy.parse_cf('huss')
q_c5_85_far   = xr.open_dataset('NorESM1-M/huss_Amon_NorESM1-M_rcp85_r1_207001-209912_2.5_anu.nc'
                               ).metpy.parse_cf('huss')

In [ ]:
# Load files CMIP6
pr_c6_hist = xr.open_mfdataset('NorESM2-LM/pr_Amon_NorESM2-LM_historical_r*_2.5_mes.nc',
                              combine='nested',
                              concat_dim='r').metpy.parse_cf('pr')
pr_c6_ens = pr_c6_hist.mean(dim='r', keep_attrs=True)
pr_c6_126_close = xr.open_dataset('NorESM2-LM/pr_Amon_NorESM2-LM_ssp126_r1_2020-2049_2.5_mes.nc'
                                 ).metpy.parse_cf('pr')
pr_c6_126_far   = xr.open_dataset('NorESM2-LM/pr_Amon_NorESM2-LM_ssp126_r1_2070-2099_2.5_mes.nc'
                                 ).metpy.parse_cf('pr')
pr_c6_585_close = xr.open_dataset('NorESM2-LM/pr_Amon_NorESM2-LM_ssp585_r1_2020-2049_2.5_mes.nc'
                                 ).metpy.parse_cf('pr')
pr_c6_585_far   = xr.open_dataset('NorESM2-LM/pr_Amon_NorESM2-LM_ssp585_r1_2070-2099_2.5_mes.nc'
                                 ).metpy.parse_cf('pr')
# """"""""""""""""""""""""""""""""""""""
ev_c6_hist = xr.open_mfdataset('NorESM2-LM/evspsbl_Amon_NorESM2-LM_historical_r*_2.5_mes.nc',
                               combine='nested',
                               concat_dim='r').metpy.parse_cf('evspsbl')
ev_c6_ens = ev_c6_hist.mean(dim='r', keep_attrs=True)
ev_c6_126_close = xr.open_dataset('NorESM2-LM/evspsbl_Amon_NorESM2-LM_ssp126_r1_2020-2049_2.5_mes.nc'
                                 ).metpy.parse_cf('evspsbl')
ev_c6_126_far   = xr.open_dataset('NorESM2-LM/evspsbl_Amon_NorESM2-LM_ssp126_r1_2070-2099_2.5_mes.nc'
                                 ).metpy.parse_cf('evspsbl')
ev_c6_585_close = xr.open_dataset('NorESM2-LM/evspsbl_Amon_NorESM2-LM_ssp585_r1i1p1f1_gn_2020-2049_2.5_mes.nc'
                                 ).metpy.parse_cf('evspsbl')
ev_c6_585_far   = xr.open_dataset('NorESM2-LM/evspsbl_Amon_NorESM2-LM_ssp585_r1i1p1f1_gn_2070-2099_2.5_mes.nc'
                                 ).metpy.parse_cf('evspsbl')
# """"""""""""""""""""""""""""""""""""""
ep_c6_hist = xr.open_mfdataset('NorESM2-LM/etp_Amon_NorESM2-LM_historical_r*_2.5_anu.nc',
                               combine='nested',
                               concat_dim='r').metpy.parse_cf('etp')
ep_c6_ens = ep_c6_hist.mean(dim='r', keep_attrs=True)
ep_c6_126_close = xr.open_dataset('NorESM2-LM/etp_Amon_NorESM2-LM_ssp126_r1_2020-2049_2.5_anu.nc'
                                 ).metpy.parse_cf('etp')
ep_c6_126_far   = xr.open_dataset('NorESM2-LM/etp_Amon_NorESM2-LM_ssp126_r1_2070-2099_2.5_anu.nc'
                                 ).metpy.parse_cf('etp')
ep_c6_585_close = xr.open_dataset('NorESM2-LM/etp_Amon_NorESM2-LM_ssp585_r1_2020-2049_2.5_anu.nc'
                                 ).metpy.parse_cf('etp')
ep_c6_585_far   = xr.open_dataset('NorESM2-LM/etp_Amon_NorESM2-LM_ssp585_r1_2070-2099_2.5_anu.nc'
                                 ).metpy.parse_cf('etp')
# """"""""""""""""""""""""""""""""""""""
t_c6_hist = xr.open_mfdataset('NorESM2-LM/tas_Amon_NorESM2-LM_historical_r*_2.5.nc',
                              combine='nested',
                              concat_dim='r').metpy.parse_cf('tas')
t_c6_ens = t_c6_hist.mean(dim='r', keep_attrs=True)
t_c6_126_close = xr.open_dataset('NorESM2-LM/tas_Amon_NorESM2-LM_ssp126_r1_2020-2049_2.5.nc'
                                ).metpy.parse_cf('tas')
t_c6_126_far   = xr.open_dataset('NorESM2-LM/tas_Amon_NorESM2-LM_ssp126_r1_2070-2099_2.5.nc'
                                ).metpy.parse_cf('tas')
t_c6_585_close = xr.open_dataset('NorESM2-LM/tas_Amon_NorESM2-LM_ssp585_r1_2020-2049_2.5.nc'
                                ).metpy.parse_cf('tas')
t_c6_585_far   = xr.open_dataset('NorESM2-LM/tas_Amon_NorESM2-LM_ssp585_r1_2070-2099_2.5.nc'
                                ).metpy.parse_cf('tas')
# """"""""""""""""""""""""""""""""""""""
q_c6_hist = xr.open_mfdataset('NorESM2-LM/huss_Amon_NorESM2-LM_historical_r*_2.5.nc',
                              combine='nested',
                              concat_dim='r').metpy.parse_cf('huss')
q_c6_ens = q_c6_hist.mean(dim='r', keep_attrs=True)
q_c6_126_close = xr.open_dataset('NorESM2-LM/huss_Amon_NorESM2-LM_ssp126_r1_2020-2049_2.5.nc'
                                ).metpy.parse_cf('huss')
q_c6_126_far   = xr.open_dataset('NorESM2-LM/huss_Amon_NorESM2-LM_ssp126_r1_2070-2099_2.5.nc'
                                ).metpy.parse_cf('huss')
q_c6_585_close = xr.open_dataset('NorESM2-LM/huss_Amon_NorESM2-LM_ssp585_r1_2020-2049_2.5.nc'
                                ).metpy.parse_cf('huss')
q_c6_585_far   = xr.open_dataset('NorESM2-LM/huss_Amon_NorESM2-LM_ssp585_r1_2070-2099_2.5.nc'
                                ).metpy.parse_cf('huss')
# """"""""""""""""""""""""""""""""""""""

In [ ]:
weights = np.cos(np.deg2rad(pr_c5_ens.lat))
media_pesada_pr = pr_c5_ens.weighted(weights)
media_pesada_pr.mean(dim=('lat', 'lon')).plot()
plt.show()
media_pesada_pr = (pr_c5_ens * (area_celda(pr_c5_ens.lat) / area_celda(pr_c5_ens.lat).mean(
                    dim='lat'))).mean(dim=('lat','lon'))
media_pesada_pr.plot()
media_pesada_ev = ((ev_c5_ens * (area_celda(ev_c5_ens.lat) / area_banda(-90, 90))).sum(dim=('lat', 'lon')))
media_pesada_ev.plot()
plt.show()
media_cos_pr = (pr_c5_ens * np.cos(np.deg2rad(pr_c5_ens.lat))).mean(dim=('lat', 'lon'))
media_cos_ev = (ev_c5_ens * np.cos(np.deg2rad(ev_c5_ens.lat))).mean(dim=('lat', 'lon'))
media_cos_pr.plot()
media_cos_ev.plot()
plt.show()
print(media_pesada_pr.mean().values)
print(media_pesada_ev.mean().values)
print((media_pesada_pr / media_cos_pr).values)

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
pr_c5_ens.mean(dim='time').plot(vmin=0, vmax=2700, transform=ccrs.PlateCarree())
ax.coastlines()
plt.show()

#plt.figure(figsize=(19,10))
ax = plt.axes(projection=ccrs.Robinson())
(ev_c5_ens.mean(dim='time')).plot(vmin=0, vmax=2700, transform=ccrs.PlateCarree())
ax.coastlines()
plt.show()

In [ ]:
pr = regrid(pr_c5_ens)
ax = plt.axes(projection=ccrs.Robinson())
pr.mean(dim='time').plot(vmin=0, vmax=2700, transform=ccrs.PlateCarree())
ax.coastlines()
plt.show()
print(pr.lat.values)

In [ ]:
#df_p_e = pd.Dataframe()

def tabla_bandas_latitud_peixoto(pr, ev):
    p_e_df = pd.DataFrame(columns=['Lat', 'P', 'E', 'P-E', 'E/P', '(P-E)/P'])
    weights = np.cos(np.deg2rad(pr.lat))
    
    def nice_lat(lat):
        if lat < 0:
            nice = str(np.abs(lat)) + '°S'
        elif lat > 0:
            nice = str(np.abs(lat)) + '°N'
        elif lat == 0:
            nice = str(np.abs(lat)) + '°'
        return nice

    for i in range(-90, 90, 10):
        precip = np.around((pr * np.cos(np.deg2rad(pr.lat))).where(pr.lat >= i
                                     ).where(pr.lat <= (i + 10)
                                              ).weighted(weights).mean(dim=('lat', 'lon', 'time')).values )
        evap = np.around((ev  * np.cos(np.deg2rad(ev.lat))).where(ev.lat >= i
                                     ).where(ev.lat < (i + 10)
                                            ).mean(dim=('lat', 'lon', 'time')).values)
        df2 = pd.DataFrame([[nice_lat(i) + ' - ' + nice_lat((i + 10)), precip,
                             evap, precip-evap,
                             evap/precip, (precip-evap)/precip]],
                       columns=['Lat', 'P', 'E', 'P-E', 'E/P', '(P-E)/P'])
        p_e_df =  p_e_df.append(df2)
    return p_e_df

print(tabla_bandas_latitud_peixoto(pr_c5_ens, ev_c5_ens))

In [ ]:
#df_p_e = pd.Dataframe()

def tabla_bandas_latitud_peixoto(pr, ev):
    p_e_df = pd.DataFrame(columns=['Lat', 'P', 'E', 'P-E', 'E/P', '(P-E)/P'])
    weights = np.cos(np.deg2rad(pr.lat))
    def nice_lat(lat):
        if lat < 0:
            nice = str(np.abs(lat)) + '°S'
        elif lat > 0:
            nice = str(np.abs(lat)) + '°N'
        elif lat == 0:
            nice = str(np.abs(lat)) + '°'
        return nice

    for i in range(-90, 90, 10):
        precip = ((pr ).where(
                  pr.lat >= i).where(pr.lat <= (i + 10)).weighted(
                  weights).mean(dim=('lat', 'lon', 'time')).values)
        evap = ((ev).where(
                ev.lat >= i).where(ev.lat <= (i + 10)).weighted(
                weights).mean(dim=('lat', 'lon', 'time')).values)
        df2 = pd.DataFrame([[nice_lat(i) + ' - ' + nice_lat((i + 10)), precip,
                             evap, precip-evap,
                             evap/precip, (precip-evap)/precip]],
                       columns=['Lat', 'P', 'E', 'P-E', 'E/P', '(P-E)/P'])
        df2['P'] = df2['P'].astype(int)
        df2['E'] = df2['E'].astype(int)
        df2['P-E'] = df2['P-E'].astype(int)
        p_e_df =  p_e_df.append(df2)
    return p_e_df

print(tabla_bandas_latitud_peixoto(regrid(pr_c5_ens), regrid(ev_c5_ens)))

In [ ]:
print(regrid(pr_c5_26_close).lat.values)
plt.figure(figsize=(20,10))
ax = plt.axes(projection=ccrs.PlateCarree())
regrid(pr_c5_26_close).where(
    regrid(pr_c5_26_close).lat < -40).where(
    regrid(pr_c5_26_close).lat >=-60).mean(
    dim='time').plot(
    transform=ccrs.PlateCarree())
regrid(pr_c5_26_close).where(
    regrid(pr_c5_26_close).lat < -20).where(
    regrid(pr_c5_26_close).lat >-40).mean(
    dim='time').plot(
    transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.show()

In [ ]:
balance_agua = pr_c5_ens - ev_c5_ens
ax = plt.axes(projection=ccrs.Robinson())
balance_agua.mean(dim='time').plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines()